In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [4]:
## MNIST -Multinomial Classification#relu#Xavier initialization 도입 초기 W값 지정#Drop out(overfitting 방지)
%time
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


tf.reset_default_graph()
#1. Data Loading
mnist = input_data.read_data_sets("./Data/mnist",one_hot = True) #one hot형태의 y 측 데이터 로딩

#2. Placeholer
##입력 데이터는 image data, 3차원 가로 세로 칼럼(depts 3 rgb color,빨노초 3레이어 지금은 흑백이어서 2차원 data.)
##2차원 데이터 이미지 데이터. 
#원래는 이미지 개수 가로 세로 칼라.-> 가로세로를 pixel data로 제공 = 이미지 데이터를 1차원으로 제공 28*28 = 784개의 열
#, 칼라도 흑백으로 생략 -> 2차원

X = tf.placeholder(shape = [None,784], dtype = tf.float32)
Y = tf.placeholder(shape = [None,10], dtype = tf.float32) #one hot 인코딩 y label

#3. Weight &bias
# W = tf.Variable(tf.random_normal([784,10]), name ="weight")
# b = tf.Variable(tf.random_normal([10]), name ="bias")
#W1 = tf.Variable(tf.random_normal([784,256]), name ="weight1")
keep = tf.placeholder(dtype = tf.float32) #입력 값으로 사용하겠다. 상수로 박지 않고. 
W1 = tf.get_variable("wight1",shape = [784,256], initializer=  tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name ="bias1")
_layer1 = tf.nn.relu(tf.matmul(X,W1)+b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep) #keepprob 유지할 확률 0.5~0.7
#W2 = tf.Variable(tf.random_normal([256,512]), name ="weight2")
W2 = tf.get_variable("wight2",shape = [256,512], initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]), name ="bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1,W2)+b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep)
#W3 = tf.Variable(tf.random_normal([512,10]), name ="weight3")
W3 = tf.get_variable("wight3",shape = [512,10], initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name ="bias3")

#4. Hypothesis
# logit = tf.matmul(X,W) + b
# H = tf.nn.sotfmax(logit) #확률값으로 결과를 얻는 방법
H = tf.matmul(layer2,W3) +b3
# H = tf.nn.softmax(logit)안해도 됨


# 5. Cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = H,labels=Y))

#6. Train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#7. Session & 초기화
sess=tf.Session()
sess.run(tf.global_variables_initializer())

#8.학습   => 이전의 for문 방법은 데이터가 작기때문에 가능했다, but 데이터가 커지면 대치처리를 해줘야 함
train_epoch = 30   #우리가 가지고 있는 데이터를 가지고 n번 학습하는 것 = n epoch
batch_size = 100   #한번에 읽어들일 데이터의 크기, 몇개씩 잘라서 들고올건가
for step in range(train_epoch):
    num_of_iter = int(mnist.train.num_examples/batch_size)  #반복횟수 : 전체데이터 / batch_size 
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)       #전체의 훈련데이터(mnist.train)에서 데이터를 100개씩 가져옴
        _, cost_val= sess.run([train, cost],feed_dict={X:batch_x,Y:batch_y,keep:0.7})
        
    if step % 3 == 0:
        print(cost_val)
        
#accuracy
predict=tf.argmax(H,1)
correct=tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("="*10)

#정확도 출력
print(sess.run(accuracy, feed_dict={X:mnist.test.images,Y:mnist.test.labels, keep:1}))

Wall time: 1 ms
Extracting ./Data/mnist\train-images-idx3-ubyte.gz
Extracting ./Data/mnist\train-labels-idx1-ubyte.gz
Extracting ./Data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./Data/mnist\t10k-labels-idx1-ubyte.gz
0.71490115
0.52129847
0.3381587
0.28896254
0.30678657
0.21497002
0.21150793
0.1287621
0.24726202
0.32844788
0.9577
